**From a series of application of modern interpolation methods for economics: written by [Mahdi E Kahou](https://sites.google.com/site/mahdiebrahimikahou/about-me)**


# Goal of this notebook  

The purpose of this notebook is to demonstrate:  
1. How to numerically calculate the expectation of a univariate function $v(x)$ with respect to a normal distribution.  
2. How to apply the Gauss–Hermite quadrature method for this task.  
3. Since these lecture notes focus on machine learning methods, we will emphasize the case where $v(x)$ is represented by a neural network.

 ---

## The Problem

Consider a function $v: \mathbb{R} \to \mathbb{R}$.  
We are interested in numerically computing  

$$
\begin{align*}
\mathbb{E}[v(X)] = \int_{-\infty}^{\infty} v(x) f(x;\mu,\sigma) dx
\end{align*}
$$

where $f(x;\mu,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$ is the pdf of a normal distribution $\mathcal{N}(\mu,\sigma^2)$.

---

## Gauss–Hermite quadrature

This method approximates the following integral by calculating a weighted sum of the function values at specific points, called **nodes**:

$$
\begin{align*}
\int_{-\infty}^{\infty} h(z) e^{-z^2} \, dz \;\approx\; \sum_{i=1}^n w_i \, h(z_i)
\end{align*}
$$

- $n$: the number of nodes  
- $w_i$: the quadrature weights  
- $z_i$: the nodes, which are the roots of the *physicists’ version* of the [Hermite polynomial](https://en.wikipedia.org/wiki/Hermite_polynomials) of degree $n$.  

---

**Important note:** Our original goal is to calculate  

$$
\begin{align*}
\frac{1}{\sqrt{2\pi\sigma^2}} \int_{-\infty}^{\infty} v(x)\, e^{-\frac{(x-\mu)^2}{2\sigma^2}} \, dx,
\end{align*}
$$  

which involves a normal distribution. The Gauss–Hermite formula above looks slightly different because it is tailored to integrals of the form  

$$
\begin{align*}
\int_{-\infty}^{\infty} h(z) e^{-z^2} dz.
\end{align*}
$$  

We will reconcile the two expressions by applying an appropriate change of variables.



## Going from $\int_{-\infty}^{\infty} h(z) e^{-z^2} \, dz$ to $\frac{1}{\sqrt{2\pi\sigma^2}} \int_{-\infty}^{\infty} v(x) e^{-\tfrac{(x-\mu)^2}{2\sigma^2}} \, dx$

Consider the change of variable  
$$
z = \frac{x - \mu}{\sqrt{2}\sigma}.
$$  

Then  
$$
x = \mu + \sqrt{2}\sigma z, \qquad dx = \sqrt{2}\sigma \, dz.
$$  

With this substitution, we can write  
$$
\frac{1}{\sqrt{2\pi\sigma^2}} \int_{-\infty}^{\infty} v(x) 
e^{-\tfrac{(x-\mu)^2}{2\sigma^2}} \, dx
= \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} 
v(\mu + \sqrt{2}\sigma z) e^{-z^2} \, dz.
$$  

Define  
$$
\tilde{v}(z) \equiv \frac{1}{\sqrt{\pi}} v(\mu + \sqrt{2}\sigma z).
$$  

Then  
$$
\mathbb{E}[v(X)] 
= \int_{-\infty}^{\infty} \tilde{v}(z) e^{-z^2} \, dz
\approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\mu + \sqrt{2}\sigma z_i),
$$  
where the last equality follows from Gauss–Hermite quadrature.

---

Let's implement this in Python:


# importing packages we need

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def Ev(v,μ,σ,n):
    #v: function, μ : mean, σ: stdev, n: number of nodes
    nodes, weights = np.polynomial.hermite.hermgauss(n)
    nodes_scaled = μ + (nodes*σ*np.sqrt(2))
    weights_scaled = weights/np.sqrt(np.pi)
    function_values = v(nodes_scaled)
    approx_mean = np.sum(function_values*weights_scaled)
    return approx_mean

## Simple examples

### Example 1: Linear functions

Consider the function  
$$
v(x) = a x + b.
$$  

Suppose $X \sim \mathcal{N}(\mu,\sigma^2)$. We want to compute the expectation $\mathbb{E}[v(X)]$. 

The closed-form solution is  
$$
\mathbb{E}[v(X)] = a \mu + b.
$$

In [3]:
def v_1(x):
    a = 2
    b = 1
    return  a*x+b

def true_mean_v_1(a,b,μ,σ):
    true_mean = a*μ + b
    print("True mean =", true_mean)

In [4]:
print("Approximate mean =", Ev(v = v_1, μ = 1, σ = 2, n = 3))

Approximate mean = 3.0


In [5]:
true_mean_v_1(a = 2,b = 1, μ = 1, σ = 2)

True mean = 3


### Example 2: Quadratic Functions

Consider the function  
$$
v(x) = a x^2 + bx + c
$$  

We want to compute the expectation $\mathbb{E}[v(X)]$. 

The closed-form solution is  
$$
E[v(X)] = a(\mu^2+\sigma^2)+ b\mu+c.
$$




In [6]:
def v_2(x):
    a = 1.1
    b = -0.1
    c = 0.8
    return (a*x**2)+(b*x)+c

def true_mean_v_2(a, b, c, μ, σ):
    true_mean =  a*(σ**2 + μ**2) + b*μ + c
    print("True mean =", true_mean)

In [7]:
print("Approximate mean =", Ev(v = v_2, μ = 0.2, σ = 0.5, n = 3))

Approximate mean = 1.0990000000000002


In [8]:
true_mean_v_2(a = 1.1, b = -0.1, c = 0.8, μ = 0.2, σ = 0.5)

True mean = 1.0990000000000002


### Example 3: A function with a kink  
Let's try something **more challenging**. 

Consider the function $v(x) = \max\{0, x\}$, with $\mu = 0$. 

The closed-form solution is  
$$
E[v(X)] = \frac{\sigma}{\sqrt{2\pi}}.
$$

In [9]:
def v_3(x):
    return np.maximum(0.0, x)

def true_mean_v_3(σ):
    true_mean =  σ/np.sqrt(2*np.pi)
    print("True mean =", true_mean)

In [10]:
print("Approximate mean =", Ev(v = v_3, μ = 0, σ = 2, n = 3))

Approximate mean = 0.5773502691896258


In [11]:
true_mean_v_3(σ = 2)

True mean = 0.7978845608028654


With nonsmooth functions, such as those with a kink, Gauss–Hermite quadrature performs poorly when using only a few nodes.

In general, the performance of quadrature methods depends on the smoothness of the function. For more details, [Art Owen's lecture notes](https://artowen.su.domains/mc/Ch-quadrature.pdf)

Let's increase the number of nodes:

In [12]:
print("Approximate mean =", Ev(v = v_3, μ = 0, σ = 2, n = 300))

Approximate mean = 0.7989796104552142


## Implementing it for a neural network
From a theoretical perspective, there is nothing special about neural networks. They are just a *function*. So one should be able to do what I just did above very easily.

The problem arises because of computational reasons. Usually the data points are in a batch, and you have to broadcast it ... fix it later

** fix this**

I have an application of this sort in mind. Consider the following example:  

We want to compute the expectation  

$$
\mathbb{E}[v(\vec{K}, x') \mid x],
$$  

where $\vec{K} \in \mathbb{R}^k$ denotes other inputs to $v$, which we treat as fixed.  

$$
x' = \rho x + \epsilon, \quad \epsilon \sim \mathcal{N}(0,\sigma^2).
$$  

Therefore, conditional on \(x\), we have  

$$
x' \mid x \sim \mathcal{N}(\rho x, \sigma^2).
$$  

For a a given $\vec{K} \in \mathbb{R}^k$ and $x\in \mathbb{R}$:

$$
\mathbb{E}[v(\vec{K}, x') \mid x] \approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}, \rho x + \sqrt{2}\sigma z_i)
$$  


When training a neural network, a `batch` of size $B$ looks like this:

$$
\text{batch} =
\begin{bmatrix}
K_1^1 & K_2^1 & \cdots & K_k^1 & x^1 \\
K_1^2 & K_2^2 & \cdots & K_k^2 & x^2 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
K_1^B & K_2^B & \cdots & K_k^B & x^B
\end{bmatrix}
$$

where $K_i^j$ shows the $i$-th element in batch $j$. 

---
**Example:** 

If you are interested in solving the stochastic version of the neo-classical growth model, see [Ljungqvist and Sargent](https://www.sfu.ca/~kkasa/Recursive_Macroeconomic_Theory_Ljungqvist_Sargent_2018.pdf), Chapter 12, Section 2. A batch in this case looks like:

$$
\text{batch} =
\begin{bmatrix}
k^1 & x^1 \\
k^2 & x^2 \\
\vdots & \vdots \\
k^B & x^B
\end{bmatrix}
$$

where $\{k^1, \dots, k^B\}$ is a subset of the capital grid, and $\{x^1, \dots, x^B\}$ is a subset of the total factor productivity grid.

---



The task ahead is to compute the expectation of $v$ for each point in the batch.  

More specifically, we want to calculate

$$
\begin{bmatrix}
\mathbb{E}[v(\vec{K}^1, x^{\prime 1}) \mid x^1] \\
\mathbb{E}[v(\vec{K}^2, x^{\prime 2}) \mid x^2] \\
\vdots \\
\mathbb{E}[v(\vec{K}^B, x^{\prime B}) \mid x^B]
\end{bmatrix}
\approx
\begin{bmatrix}
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^1, \rho x^1 + \sqrt{2}\sigma z_i) \\
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^2, \rho x^2 + \sqrt{2}\sigma z_i) \\
\vdots \\
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^B, \rho x^B + \sqrt{2}\sigma z_i)
\end{bmatrix}
$$

Let's do some matrix manipulation. Define  

$$
\zeta_i^j \equiv \rho x^j + \sqrt{2}\sigma z_i,
$$

where $x^j$ represents the $x$ in batch $j$, and $z_i$ corresponds to node $i$. Then we can rewrite:

$$
\begin{bmatrix}
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^1, \rho x^1 + \sqrt{2}\sigma z_i) \\
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^2, \rho x^2 + \sqrt{2}\sigma z_i) \\
\vdots \\
\frac{1}{\sqrt{\pi}} \sum_{i=1}^n w_i \, v(\vec{K}^B, \rho x^B + \sqrt{2}\sigma z_i)
\end{bmatrix}
=
\begin{bmatrix}
v(\vec{K}^1, \zeta^1_1) & v(\vec{K}^1, \zeta^1_2) & \cdots & v(\vec{K}^1, \zeta^1_n) \\
v(\vec{K}^2, \zeta^2_1) & v(\vec{K}^2, \zeta^2_2) & \cdots & v(\vec{K}^2, \zeta^2_n) \\
\vdots & \vdots & \ddots & \vdots \\
v(\vec{K}^B, \zeta^B_1) & v(\vec{K}^B, \zeta^B_2) & \cdots & v(\vec{K}^B, \zeta^B_n)
\end{bmatrix}_{B\times n}
\begin{bmatrix}
\frac{w_1}{\sqrt{\pi}}\\
\frac{w_2}{\sqrt{\pi}}\\
\vdots\\
\frac{w_n}{\sqrt{\pi}}
\end{bmatrix}.
$$

The new matrix above has dimensions $B \times n$, but the total number of inputs is $B \times (k+1) \times n$. Why?  
- $B$: number of batches  
- $k+1$: dimensionality of a single batch  
- $n$: number of nodes used for Gauss–Hermite quadrature

So we have to resort to **tensors**


The tensor is going to contain 

$$
\begin{align*}
\begin{bmatrix}
\vec{K}^1& \zeta^1_1 \\
\vec{K}^2& \zeta^2_1  \\
\vdots & \vdots \\
\vec{K}^B& \zeta^B_1 
\end{bmatrix}_{B \times (k+1)}, 
\begin{bmatrix}
\vec{K}^1& \zeta^1_2 \\
\vec{K}^2& \zeta^2_2  \\
\vdots & \vdots \\
\vec{K}^B& \zeta^B_2 
\end{bmatrix}_{B \times (k+1)}, \cdots \begin{bmatrix}
\vec{K}^1& \zeta^1_n \\
\vec{K}^2& \zeta^2_n  \\
\vdots & \vdots \\
\vec{K}^B& \zeta^B_n 
\end{bmatrix}_{B \times (k+1)}
\end{align*}
$$

---

Let's implement it in Python:

In [125]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [127]:
n = 7
nodes, weights = np.polynomial.hermite.hermgauss(n)
node_tensor = torch.tensor(nodes, dtype=torch.float32)
weight_adjusted_tensor = (1/np.sqrt(np.pi))*torch.tensor(weights, dtype=torch.float32).unsqueeze(1)

In [128]:
torch.manual_seed(0)  # Fix the random seed

B = 3
K = torch.linspace(start=0, end=1, steps=B).unsqueeze(1) #k=1
x = torch.rand(B).unsqueeze(1)
batch = torch.cat((K, x), dim=1)
batch

tensor([[0.0000, 0.4963],
        [0.5000, 0.7682],
        [1.0000, 0.0885]])

In [129]:
node_tensor

tensor([-2.6520, -1.6736, -0.8163,  0.0000,  0.8163,  1.6736,  2.6520])

In [130]:
ρ = 0.95
σ = 0.5

In [131]:
ζ = (ρ*x) + (np.sqrt(2)*σ*nodes) # Gives a B by n matrix
ζ

tensor([[-1.4038, -0.7119, -0.1058,  0.4714,  1.0486,  1.6548,  2.3467],
        [-1.1454, -0.4536,  0.1526,  0.7298,  1.3070,  1.9132,  2.6050],
        [-1.7912, -1.0993, -0.4931,  0.0841,  0.6613,  1.2674,  1.9593]],
       dtype=torch.float64)

In [132]:
test = torch.zeros(n,B,2) # n, B, k+1
test[:] = K
test[:,:,1] = ζ.T

In [133]:
class NN(nn.Module):
    def __init__(self,
                 dim_hidden = 128,
                layers = 1,
                hidden_bias = True):
        super().__init__()
        self.dim_hidden= dim_hidden
        self.layers = layers
        self.hidden_bias = hidden_bias

        torch.manual_seed(123)
        module = []
        module.append(nn.Linear(2,self.dim_hidden, bias = self.hidden_bias))
        module.append(nn.Tanh())

        for i in range(self.layers-1):
            module.append(nn.Linear(self.dim_hidden,self.dim_hidden, bias = self.hidden_bias))
            module.append(nn.Tanh())

        module.append(nn.Linear(self.dim_hidden,1))
        module.append(nn.Softplus(beta = 1.0)) 

        self.q = nn.Sequential(*module)


    def forward(self, x):
        out = self.q(x) # first element is consumption, the second element is capital
        return  out

In [134]:
model = NN()

In [136]:
E_res = model(test).squeeze().T@weight_adjusted_tensor
E_res

tensor([[0.7358],
        [0.7489],
        [0.7180]], grad_fn=<MmBackward0>)

In [123]:
## Testing with Monte-carlo

In [137]:
k_1 = batch[0,0]
x_1 = batch[0,1]
mean_1 = ρ*x_1


# 10 random draws
n_draws = 1000000
samples_x_1 = torch.normal(mean=mean_1, std=σ, size=(n_draws,))
batch_1 = torch.zeros([n_draws,2])
batch_1[:,0] = k_1
batch_1[:,1] = samples_x_1
model(batch_1).mean()

tensor(0.7358, grad_fn=<MeanBackward0>)

In [138]:
k_2 = batch[1,0]
x_2 = batch[1,1]
mean_2 = ρ*x_2


# 10 random draws
n_draws = 1000000
samples_x_2 = torch.normal(mean=mean_2, std=σ, size=(n_draws,))
batch_2 = torch.zeros([n_draws,2])
batch_2[:,0] = k_2
batch_2[:,1] = samples_x_2
model(batch_2).mean()

tensor(0.7489, grad_fn=<MeanBackward0>)

In [139]:
k_3 = batch[2,0]
x_3 = batch[2,1]
mean_3 = ρ*x_3


# 10 random draws
n_draws = 1000000
samples_x_3 = torch.normal(mean=mean_3, std=σ, size=(n_draws,))
batch_3 = torch.zeros([n_draws,2])
batch_3[:,0] = k_3
batch_3[:,1] = samples_x_3
model(batch_3).mean()

tensor(0.7180, grad_fn=<MeanBackward0>)